## Import thư viện

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.common.exceptions import  NoSuchElementException,ElementClickInterceptedException
import pandas as pd
from time import sleep
import os
import re 

## Cấu hình tùy chọn cho Edge

In [3]:
options = Options()
options.add_argument("--start-maximized")  # Mở ở chế độ toàn màn hình
options.add_argument("--inprivate")  # Chế độ ẩn danh (InPrivate mode)

## Đăng nhập

In [4]:
driver = webdriver.Edge(options=options)
driver.get("https://www.facebook.com/groups/ued.confessions?locale=vi_VN")

In [5]:
try:
    username = driver.find_element(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .xod5an3 input')
    password = driver.find_elements(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .x1c436fg input')[0]
    login = driver.find_elements(By.CSS_SELECTOR, '.x1egiwwb.x4l50q0 .x1c436fg')[1]
except:
    username = driver.find_element(By.CSS_SELECTOR, 'input[name = "email" and type = "email"]')
    password = driver.find_element(By.CSS_SELECTOR, 'input[name = "pass" and type = "password"]')
    login = driver.find_element(By.CSS_SELECTOR, 'div[aria-label = "Đăng nhập"]') 

sleep(2) 

username.send_keys('khoalolriot@gmail.com')
password.send_keys('Akhoa@6204')
login.click()

## Hàm scroll

In [6]:
def scroll():
    # Lấy chiều cao màn hình (viewport height) từ trình duyệt
    viewport_height = driver.execute_script("return window.innerHeight;")
    
    # Cuộn xuống một đoạn bằng chiều cao màn hình
    driver.execute_script(f"""
        window.scrollBy({{
            top: {viewport_height}, 
            behavior: 'smooth'
        }});
    """)
    sleep(2)  # Chờ 5 giây sau khi cuộn xuống
    
    # Cuộn lên lại một đoạn bằng chính chiều cao đã cuộn xuống
    driver.execute_script(f"""
        window.scrollBy({{
            top: {-viewport_height},
            behavior: 'smooth'
        }});
    """)
  # Chờ sau khi cuộn lên  
def scroll_to_post(post, time):
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", post)
    sleep(time)

## Đường dẫn lưu file

In [7]:
path_normal = 'D:/kpw/crawlFB/post_facebook.csv'
path_reel = 'D:/kpw/crawlFB/post_reel.csv'

## Lưu kết quả

In [8]:
def saveOrCreate(result_post_normal, result_post_reel):
    # Kiểm tra và tạo file CSV cho post_facebook nếu không tồn tại
    if not os.path.exists(path_normal):
        os.makedirs(os.path.dirname(path_normal), exist_ok=True)
        df_normal = pd.DataFrame(columns=['index', 'linkPoster', 'name', 'role', 'time', 'content', 'content_img', 'react_list', 'comment'])
        df_normal.to_csv(path_normal, index=False)

    # Nếu có dữ liệu mới, thêm vào file post_facebook.csv
    if result_post_normal:
        new_post_normal = pd.DataFrame(result_post_normal)
        df_normal = pd.read_csv(path_normal)
        df_normal = pd.concat([df_normal, new_post_normal], ignore_index=True)
        df_normal.to_csv(path_normal, index=False)

    # Kiểm tra và tạo file CSV cho post_reel nếu không tồn tại
    if not os.path.exists(path_reel):
        os.makedirs(os.path.dirname(path_reel), exist_ok=True)
        df_reel = pd.DataFrame(columns=['index', 'linkPoster', 'name', 'time', 'content', 'linkVideo', 'typeVideo'])
        df_reel.to_csv(path_reel, index=False)

    # Nếu có dữ liệu mới, thêm vào file post_reel.csv
    if result_post_reel:
        new_post_reel = pd.DataFrame(result_post_reel)
        df_reel = pd.read_csv(path_reel)
        df_reel = pd.concat([df_reel, new_post_reel], ignore_index=True)
        df_reel.to_csv(path_reel, index=False)


## Lấy ra post đầu tiên của lần crawl trước

In [9]:
def get_first_post():
    # Đọc dữ liệu từ các file CSV
    df_normal = pd.read_csv(path_normal)
    df_reel = pd.read_csv(path_reel)

    # Kiểm tra nếu cả hai dataframe đều rỗng
    if df_normal.empty and df_reel.empty:
        return None
    
    # Nếu df_normal không rỗng
    if not df_normal.empty:
        first_row_normal = df_normal.iloc[0]
    else:
        first_row_normal = None

    # Nếu df_reel không rỗng
    if not df_reel.empty:
        first_row_reel = df_reel.iloc[0]
    else:
        first_row_reel = None

    # So sánh và trả về dòng có index nhỏ hơn nếu cả hai dataframe không rỗng
    if first_row_normal is not None and first_row_reel is not None:
        if first_row_normal['index'] > first_row_reel['index']:
            result = first_row_reel
        else:
            result = first_row_normal
    elif first_row_normal is not None:
        result = first_row_normal
    elif first_row_reel is not None:
        result = first_row_reel
    else:
        result = None

    return result


## Sắp xếp Post mới nhất

In [10]:
chooseList = driver.find_element(By.CSS_SELECTOR, '[role="feed"] div:first-child div div div div')
chooseList.click()
sleep(1)

choice = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[2]/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div[1]/div/div[3]')
choice.click()

# Extract

In [11]:
def extractInfo(info_box):
    try:
        user_link = info_box.find_element(By.XPATH, './div/div[2]/div/div[1]/span/div/h2/span/span/a').get_attribute('href')
    except:
        user_link = None
    user_name = info_box.find_element(By.XPATH, './div/div[2]/div/div[1]').find_element(By.CSS_SELECTOR, '.xt0psk2').text  
    time_post = info_box.find_element(By.XPATH, './div/div[2]/div/div[2]')

    try: 
        role = time_post.find_element(By.CSS_SELECTOR,'.x3nfvp2.x1kgmq87').text
    except NoSuchElementException:
        role = None

    try: 
        time_link = time_post.find_element(By.CSS_SELECTOR, 'use').get_attribute('xlink:href')
        time_post = driver.find_element(By.CSS_SELECTOR, f'text{time_link}').get_attribute('textContent')
    except NoSuchElementException:  
        time_post = time_post.text.replace('·','').strip()
    return user_link, user_name, time_post, role

In [12]:
def extractContent(content_box):
    # Lấy thông tin bài đăng
    try: # Có nội dung
        main_content = content_box.find_element(By.XPATH, './div[1]')
        try: 
            more_content = main_content.find_element(By.CSS_SELECTOR, 'div[role="button"]')
            if more_content.text.strip() == 'Xem thêm': 
                try:
                    scroll_to_post(more_content, 0)
                    more_content.click()
                except ElementClickInterceptedException: 
                    driver.execute_script("window.scrollBy({top: -400, behavior: 'smooth'});")
                    try:
                        more_content.click()
                    except ElementClickInterceptedException:
                        pass
        except NoSuchElementException: 
            pass
        main_content = content_box.find_element(By.XPATH, './div[1]').text
    except NoSuchElementException: # Không có nội dung
        main_content = None

    # Lấy ảnh trong bài đăng
    content_img_post = {}        
    try: # Có ảnh  
        content_img = content_box.find_element(By.XPATH, './div[2]')
        content_img_list = content_img.find_elements(By.CSS_SELECTOR, 'img')
        for i, img in enumerate(content_img_list):
            content_img_post[i] = img.get_attribute('src')
    except NoSuchElementException: # Không có ảnh
        pass
    return main_content, content_img_post

In [13]:
def extractReact(react_cmt_box):
    react_list = {
        "Thích" : 0,
        'Yêu thích' : 0,
        'Thương thương': 0,
        'Haha': 0,
        'Wow': 0,
        'Buồn': 0,
        'Phẫn nộ': 0
    }
    try: # Bài đăng có người thả cảm xúc
        react = react_cmt_box.find_element(By.XPATH,'./div/div/div[1]')

        react_icon = react.find_elements(By.XPATH,'./div/div[1]/div/div[1]/span/span/span')
        for react in react_icon: 
            rs = react.find_element(By.CSS_SELECTOR,'div[aria-label]').get_attribute('aria-label').split(':')[0].strip()
            for i in react_list: 
                if i== rs:
                    react_list[i] = react.find_element(By.CSS_SELECTOR,'div[aria-label]').get_attribute('aria-label').split(':')[1].strip().split()[0]
    except NoSuchElementException: # Bài đăng không có người thả cảm xúc
        pass
    
    return react_list
    

In [14]:
def extractComment_popUp(post, post_data_cmt):
    cmt_box = post.find_element(By.XPATH, './div/div/div/div[2]/div[2]/div/div/div[2]/div/div[2]/div[3]')
    scroll_to_post(cmt_box,0)
    cmt_list = cmt_box.find_elements(By.XPATH, './div[position() < last()]')
    for cmt in cmt_list:
        # Nhấn tất cả các nút xem thêm của comment đó
        while True:
            try:
                more_cmt_button_list = cmt.find_elements(By.CSS_SELECTOR, 'div[role="button"]')
                has_clicked = False 
                for button in more_cmt_button_list:
                    if "Xem" in button.text and "phản hồi" in button.text:
                        try:    
                            scroll_to_post(button, 0)
                            button.click()
                        except: 
                            driver.execute_script("window.scrollBy({top: -200, behavior: 'smooth'});")
                            button.click()
                        has_clicked = True
                        break  
                if not has_clicked:
                    break
            except Exception:
                break
        results = []
        content = cmt.text.split('\n')
        # Tiền xử lí comment
        try:
            i = 0
            for index_, line in enumerate(content):  
                if line == 'Thích': 
                    cmt_sub = content[i: index_ - 1]
                    results.append(cmt_sub)
                if line == "Chia sẻ" and content[index_ + 1] != 'Đã chỉnh sửa':
                    i = index_ + 1

                if line == 'Đã chỉnh sửa' and content[index_ + 1].isdigit():
                    i = index_ + 2
                elif line == 'Đã chỉnh sửa' and re.search(r'[a-zA-Z]', content[index_ + 1]):
                    i = index_ + 1 
                if line == 'Viết phản hồi công khai…':
                    i = index_ + 1        
        except: pass  
        # Lưu người viết và nội dung comment 
        for result in results: 
            for index_, line in enumerate(result):
                if index_ == 0: 
                    if line.isdigit():
                        name = result[index_ + 1]
                    else: 
                        name = result[index_]
                        num = index_
                content = ' '.join(result[num + 1: ])
                if line == '  ·' and result[index_ + 1] == 'Theo dõi':
                    content = ' '.join(result[index_ + 2 :])
                    break
                
            comment = {
                'name' : name, 
                'content': content
            }
            post_data_cmt.append(comment)
    check_moreCmt = False
    close = post.find_element(By.XPATH, './div/div/div/div[1]/div/div[3]')
    close.click()
    return check_moreCmt

In [15]:
def extractComment(react_cmt_box, check):
    cmt_box = react_cmt_box.find_element(By.XPATH, './div/div')
    cmt_list = cmt_box.find_elements(By.XPATH,'./div')
    check_moreCmt = check if check else False
    post_data_cmt = []
    try:
        if len(cmt_list) < 4: # Không có comment nào trong bài đăng
            pass 
        else:
            more_button = cmt_box.find_element(By.XPATH, './div[3]')
            if 'Xem thêm' in more_button.text or 'Xem tất cả' in more_button.text:
                try:
                    scroll_to_post(more_button, 0)
                    more_button.click()
                except: 
                    driver.execute_script("window.scrollBy({top: -200, behavior: 'smooth'});")
                    more_button.click()
                check_moreCmt = True
    except:
        pass
            
    sleep(2)

    while True:
        if check_moreCmt:
            post = driver.find_element(By.CSS_SELECTOR, 'div[role = "dialog"]')
            check_moreCmt = extractComment_popUp(post, post_data_cmt)
        else: 
            cmt_list = react_cmt_box.find_elements(By.XPATH, './div/div/div[position() > 2 and position() < last()]')
            for cmt in cmt_list:          
                try:  
                    more_buttons = cmt.find_elements(By.CSS_SELECTOR, 'div[role = "button"]')
                    for button in more_buttons:
                        if 'Xem' in button.text and 'phản hồi' in button.text:
                            try:
                                scroll_to_post(button, 0)
                                button.click()
                            except: 
                                driver.execute_script("window.scrollBy({top: -200, behavior: 'smooth'});")
                                button.click()
                            check_moreCmt = True
                            break
                except: pass
            if not check_moreCmt: 
                for cmt in cmt_list:  
                    name = cmt.find_element(By.XPATH,'./div/div/div[2]/div[1]/div[1]/div/div/span').text
                    content = cmt.find_element(By.XPATH,'./div/div/div[2]/div[1]/div[1]/div/div/div').text
                    comment = {
                        'name' : name, 
                        'content': content
                    }
                    post_data_cmt.append(comment)
        if check_moreCmt:   
            continue 
        else: 
            break 
    return post_data_cmt


## DEBUG

In [16]:
def debug_cmt(react_cmt_box):
    try: 
        post = driver.find_element(By.CSS_SELECTOR, 'div[role = "dialog"]')
        close = post.find_element(By.CSS_SELECTOR, 'div[aria-label="Đóng"]')
        close.click()
    except:
        pass 
    post_data_cmt = []
    check_moreCmt = False
    cmt_list = react_cmt_box.find_elements(By.XPATH, './div/div/div[position() > 2 and position() < last()]')
    for cmt in cmt_list:          
        try:  
            more_buttons = cmt.find_elements(By.CSS_SELECTOR, 'div[role = "button"]')
            for button in more_buttons:
                if 'Xem' in button.text and 'phản hồi' in button.text:
                    try:
                        scroll_to_post(button, 0)
                        button.click()
                    except: 
                        driver.execute_script("window.scrollBy({top: -200, behavior: 'smooth'});")
                        button.click()
                    check_moreCmt = True
                    break
        except: pass
    try:
        post_data_cmt = extractComment(react_cmt_box, check_moreCmt)
    except: 
        for cmt in cmt_list:  
            name = cmt.find_element(By.XPATH,'./div/div/div[2]/div[1]/div[1]/div/div/span').text
            content = cmt.find_element(By.XPATH,'./div/div/div[2]/div[1]/div[1]/div/div/div').text
            comment = {
                'name' : name, 
                'content': content
            }
            post_data_cmt.append(comment)
    return post_data_cmt

## Main

In [ ]:
saveOrCreate(None, None) # Tạo csv
result_post_normal = []
result_post_reel = []
index = 0
post_list = driver.find_elements(By.CSS_SELECTOR, '[role="feed"] .x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
prev_result = get_first_post()
while True:
    # Lưu kết quả sau mỗi vòng lặp
    post_data_normal = {}
    post_data_reel = {} 
    
    # Tắt cảnh báo
    try:
        close = driver.find_element(By.XPATH,'//*[@id="facebook"]/body/div[6]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[3]')
        close.click()
        sleep(5)
    except NoSuchElementException: 
        pass  
    
    # Scroll mỗi khi crawl hết trong danh sách có sẵn
    if index >= len(post_list):  
        scroll()
        post_list_new = driver.find_elements(By.CSS_SELECTOR, '[role="feed"] .x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
        if len(post_list_new) == len(post_list):
            print()
            print('Thu thập được:',len(post_list),'bài đăng')
            break 
        else:
            post_list = post_list_new 
           
    # Scroll to post
    scroll_to_post(post_list[index], 2)

    # Main

    try: # Post normal 
        print(f"Đã scroll tới bài đăng số {index + 1}:")
        print('post normal')
        
        # Lấy Xpath của bài post 
        main  = post_list[index].find_element(By.XPATH,'./div/div/div/div/div/div/div/div/div/div[13]/div/div')
        
        # Chia nhỏ Xpath thành 3 phần
        info_box  = main.find_element(By.XPATH,'./div[2]')            
        content_box = main.find_element(By.XPATH,'./div[3]')
        react_cmt_box = main.find_element(By.XPATH,'./div[4]')
        
        # Lấy thông tin người đăng
        user_link, user_name, time_post, role = extractInfo(info_box)
        
        # Lấy thông tin bài đăng
        main_content, content_img_post = extractContent(content_box)
        
        # Lấy thông tin lượt người thả cảm xúc
        react_list = extractReact(react_cmt_box)
        
        # Lấy thông tin comment của bài đăng
        try:
            post_data_cmt = extractComment(react_cmt_box, None)
        except:
            post_data_cmt = debug_cmt(react_cmt_box) 
  
        # Lưu thông tin toàn bộ bài post 
        post_data_normal = {
            'index': index + 1,
            'linkPoster': user_link if user_name != 'Người tham gia ẩn danh' else 'No link',
            'name': user_name,
            'role': role if role else 'Thành viên',
            'time': time_post,
            'content': main_content if main_content else 'No content',
            'content_img': content_img_post if content_img_post else 'No picture',
            'react_list': react_list,
            'comment': post_data_cmt if post_data_cmt else 'No comment'
        }
        print(post_data_normal)
        index += 1
    except NoSuchElementException: # video
        print(f"Đã scroll tới bài đăng số {index + 1}:")
        print('post video')
        
        # Lấy ra Xpath cuả bài post
        main = post_list[index].find_element(By.XPATH,'./div/div/div/div/div/div/div/div/div/div[13]/div/div')
        content_box = main.find_element(By.XPATH,'./div[2]')
        
        # Thông tin bài post
        link = content_box.find_element(By.CSS_SELECTOR, 'a')
        href = link.get_attribute('href')
        
        box = link.find_element(By.XPATH,'./div[1]/div[3]/div/div/div[1]/div/div/div/div/div/div[2]/span/span')
        
        linkPoster = box.find_element(By.CSS_SELECTOR,'a').get_attribute('href')
        
        box = box.text.split('·')
        typeVideo = box[0].replace('\n', '').strip()
        name = box[1].replace('\n', '').strip()
        time = box[2].replace('\n', '').strip()
        
        # Lấy ra nội dung bài post
        try: # Có nội dung
            main_content = link.find_element(By.XPATH,'./div[1]/div[2]/div/div/div[2]/span/div')
            try: 
                more_content = main_content.find_element(By.CSS_SELECTOR, 'div[role="button"]')
                if more_content.text.strip() == 'Xem thêm': 
                    scroll_to_post(more_content, 0)
                    more_content.click()
            except NoSuchElementException: 
                pass
            main_content = link.find_element(By.XPATH, './div[1]/div[2]/div/div/div[2]/span/div').text
        except NoSuchElementException: # Không có nội dung
            main_content = None
        
        # Lưu thông tin bài post
        post_data_reel = {
            'index': index + 1,
            'linkPoster': linkPoster,
            'name': name,
            'time': time,
            'content': main_content if main_content else 'No content',
            'linkVideo': href,
            'typeVideo': typeVideo,
        }
        
        index += 1
        print(post_data_reel)
    # Kiểm tra trước đó đã crawl url này hay chưa
    if prev_result is None: # Chưa crawl
        pass
    else: # Đã crawl
        if prev_result == post_data_normal: #Tới được bài đăng đầu tiên của đợt crawl trước
            break
        if prev_result == post_data_reel: #Tới được bài đăng đầu tiên của đợt crawl trước
            break
    # Thêm kết quả vào danh sách
    if post_data_normal:  
        result_post_normal.append(post_data_normal)
    if post_data_reel:
        result_post_reel.append(post_data_reel)
    
print('Crawl thành công')


Đã scroll tới bài đăng số 1:
post normal
{'index': 1, 'linkPoster': 'No link', 'name': 'Người tham gia ẩn danh', 'role': 'Thành viên', 'time': '44 phút', 'content': 'Em chào a/c ạ\na/c cho e xin ít kinh nghiệm đăng ký tín chỉ học kỳ 2 ngành giáo dục tiểu học với , e sinh viên năm nhất ạ\nHelp me', 'content_img': 'No picture', 'react_list': {'Thích': '2', 'Yêu thích': 0, 'Thương thương': 0, 'Haha': 0, 'Wow': 0, 'Buồn': 0, 'Phẫn nộ': 0}, 'comment': 'No comment'}
Đã scroll tới bài đăng số 2:
post normal
{'index': 2, 'linkPoster': 'No link', 'name': 'Người tham gia ẩn danh', 'role': 'Thành viên', 'time': '1 giờ', 'content': 'nghiêm túc ah E năm 2 đại học, hiện tại gia đình khó khăn không đủ xiên để chi trả học phí, em chưa muốn bỏ học, e đi xin việc nhưng ngta k nhận vì ng nhỏ, 1 tờ 1h thôi ạ. e đang cần gấp lên h gặp luôn cg dc ạ, em dùng tele gram lên ai quan tâm thì nt qua đó ạ tele: @Thuyha2000\nmong ad duyệt bài', 'content_img': {0: 'https://scontent.fdad3-5.fna.fbcdn.net/v/t39.30808-

## Save

In [ ]:
saveOrCreate(result_post_normal, result_post_reel)
print('Save thành công')